# Visualization

When it comes to visualizing geospatial data with/on maps with Python, a great number of tools and techniques
are available. In this lesson we will explore several of these:

* Folium
* [ipyleaflet](https://ipyleaflet.readthedocs.io) - Interactive maps in the Jupyter notebook
* Bokeh

## Bokeh

See also:

* https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html
* [Binder for Geographic Plots in Bokeh](https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F09%20-%20Geographic%20Plots.ipynb)
* https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
* https://pythonawesome.com/bokeh-plotting-backend-for-pandas-and-geopandas/


In [22]:

from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook, show
output_notebook()


Loading BokehJS ...

If you show the figure, you can then use the wheel zoom and pan tools to navigate over any zoom level, 
and Bokeh will request the appropriate tiles from the server and insert them at the correct locations in the plot:


In [30]:

# When using in standard Python env
# output_file("tile.html")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(tools='pan, wheel_zoom', x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)



### Plotting Points on the Map

We will try to plot populated places on the above map.


In [31]:

import geopandas as gpd

# Read the data (already in Web Mercator projection
points = gpd.read_file('../data/populated_places.3857.gpkg')


In [32]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y


In [33]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)


In [34]:
points.head(5)


,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,DIFFASCII,NAMEASCII,ADM0CAP,...,name_nl,name_pl,name_sv,name_tr,name_vi,wdid_score,ne_id,geometry,x,y
0,8,10,3,Admin-0 capital,Vatican City,None,None,0,Vatican City,1.0,...,Vaticaanstad,Watykan,Vatikanstaten,Vatikan,Thành Vatican,4,1159127243,POINT (1386304.648838062 5146502.575862345),1.386305e+06,5.146503e+06
1,7,20,0,Admin-0 capital,San Marino,None,None,0,San Marino,1.0,...,San Marino,San Marino,San Marino,San Marino,Thành phố San Marino,4,1159146051,POINT (1385011.518533259 5455558.186449202),1.385012e+06,5.455558e+06
2,7,20,0,Admin-0 capital,Vaduz,None,None,0,Vaduz,1.0,...,Vaduz,Vaduz,Vaduz,Vaduz,Vaduz,4,1159146061,POINT (1059390.799771928 5963928.57622693),1.059391e+06,5.963929e+06
3,6,30,8,Admin-0 capital alt,Lobamba,None,None,0,Lobamba,0.0,...,Lobamba,Lobamba,Lobamba,Lobamba,Lobamba,4,1159146343,POINT (3473167.79100483 -3056995.457303667),3.473168e+06,-3.056995e+06
4,6,30,8,Admin-0 capital,Luxembourg,None,None,0,Luxembourg,1.0,...,Luxemburg,Luksemburg,Luxemburg,Lüksemburg,Luxembourg,4,1159146437,POINT (682388.790950537 6379291.915456847),6.823888e+05,6.379292e+06


In [35]:
p_df = points.drop('geometry', axis=1).copy()
p_df.head(2)


,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,DIFFASCII,NAMEASCII,ADM0CAP,...,name_ko,name_nl,name_pl,name_sv,name_tr,name_vi,wdid_score,ne_id,x,y
0,8,10,3,Admin-0 capital,Vatican City,None,None,0,Vatican City,1.0,...,바티칸 시국,Vaticaanstad,Watykan,Vatikanstaten,Vatikan,Thành Vatican,4,1159127243,1.386305e+06,5.146503e+06
1,7,20,0,Admin-0 capital,San Marino,None,None,0,San Marino,1.0,...,산마리노,San Marino,San Marino,San Marino,San Marino,Thành phố San Marino,4,1159146051,1.385012e+06,5.455558e+06


In [36]:
from bokeh.models import ColumnDataSource
psource = ColumnDataSource(p_df)


In [37]:
# p = figure(title="A map of populated places from a GeoPackage")
p.circle('x', 'y', source=psource, color='red', size=10)
show(p)



## Folium

Whenever you visit website that has some kind of interactive map, it
is quite probable that you are witnessing a map that has been made with
a JavaScript library called [Leaflet](http://leafletjs.com).  The
other popular library one that you might encounter is
[OpenLayers](https://openlayers.org).

There is a Python module called
[Folium](https://github.com/python-visualization/folium) that makes
it possible to visualize data that has been manipulated in Python on an
interactive Leaflet map.

### Basics

We will start with the most minimal map using the default OpenStreetMap base map.
See [Folium Quickstart](https://python-visualization.github.io/folium/quickstart.html).


In [ ]:
import folium

m = folium.Map(location=[44.43225, 26.10626])


To display it in a Jupyter notebook, simply ask for the object representation:


In [ ]:
m


You could even save this map to a file and serve it via a webserver: 

`m.save('index.html')`


### GeoJSON Overlay

It gets interesting when you can overlay the map with data manipulated
via Python. Here we overlay the map with the Polygons of all countries, though
that set is in a lower resolution clearly.


In [ ]:

countries = f'../data/countries.json'

the_map = folium.Map(
    location=[44.43225, 26.10626],
    zoom_start=2  
)

folium.GeoJson(
    countries,
    name='countries'
).add_to(the_map)

folium.LayerControl().add_to(the_map)

the_map


---
[<- Data Processing and Analysis](04-data-processing-analysis.ipynb) | [Metadata ->](06-metadata.ipynb)